# Detect particles in BSE images and EBSD intensity maps

Håkon Wiik Ånes (hakon.w.anes@ntnu.no)

In [1]:
# Switch to interactive Matplotlib backend (e.g. qt5) for control point determination
%matplotlib qt5

from datetime import date
import gc
import importlib_metadata
import os

from mapregions import MapRegions
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage
import skimage.color as skc
import skimage.exposure as ske
import skimage.io as ski
import skimage.filters as skf
from skimage.segmentation import watershed
import skimage.transform as skt


# Directories for input and output
sample = "0s"
dset_no = 1
dir_data = f"/home/hakon/phd/data/p/prover/{sample}/{dset_no}"
dir_bse = os.path.join(dir_data, "bse")
dir_imreg = os.path.join(dir_data, "imreg")
dir_kp = os.path.join(dir_data, "kp")
dir_partdet = os.path.join(dir_data, "partdet")

# Matplotlib configuration and keyword arguments
plt.rcParams.update({"axes.grid": False, "figure.figsize": (15, 5), "font.size": 12})
savefig_kw = dict(bbox_inches="tight", pad_inches=0, transparent=True, dpi=300)
scatter_kw = dict(s=70, linewidth=2, facecolor="none", clip_on=False)
#plt.ioff()

print(sample, dset_no)
print("Run date: ", date.today())
print("\nSoftware versions\n-----------------")
for pkg in ["mapregions", "matplotlib", "numpy", "orix", "scipy", "scikit-image"]:
    if pkg == "numpy":
        ver = np.__version__
    else:
        ver = importlib_metadata.version(pkg)
    print(pkg, ":", ver)

0s 1
Run date:  2022-09-07

Software versions
-----------------
mapregions : 0.1.dev1
matplotlib : 3.5.2
numpy : 1.22.4
orix : 0.9.0.post0
scipy : 1.9.0
scikit-image : 0.19.3


Step sizes (`bse` is upscaled)

In [2]:
step_size = dict(ebsd=0.1, bse=0.025, bse_full=1 / 39.2)

Dataset specific parameters

In [3]:
# Remove zero-regions resulting from stitching of BSE images
crop_bse = {
    "0s": {
        1: (slice(0, -1), slice(0, -1)),
        2: (slice(0, -1), slice(0, -1)),
        3: (slice(0, -1), slice(0, -1)),
    },
    "175c": {
        1: (slice(107, -1), slice(273, -1)),
        2: (slice(117, -1), slice(0, -1)),
        3: (slice(224, 3670), slice(251, -1)),
    },
    "300c": {
        1: (slice(0, 3826), slice(0, -1)),
        2: (slice(0, 3418), slice(0, -1)),
        3: (slice(16, 3916), slice(0, -1)),
    },
    "325c": {
        1: (slice(0, -1), slice(0, -1)),
        2: (slice(0, 3277), slice(0, -1)),
        3: (slice(129, 3745), slice(0, -1)),
    },
}
# EBSD
ebsd_slice = {
    "0s": {
        1: (slice(392, 3272), slice(1367, 4357)),
        2: (slice(4, 5364), slice(229, 5229)),
        3: (slice(190, 4390), slice(787, 4487)),
    },
    "175c": {
        1: (slice(0, 3642), slice(0, 3214)),
        2: (slice(0, 4386), slice(205, 4206)),
        3: (slice(0, -1), slice(0, 3354)),
    },
    "300c": {
        1: (slice(175, -1), slice(850, 4450)),
        2: (slice(398, -1), slice(1276, 4376)),
        3: (slice(0, -1), slice(263, 3763)),
    },
    "325c": {
        1: (slice(198, 3788), slice(229, 3930)),
        2: (slice(572, -1), slice(1083, 4083)),
        3: (slice(0, -1), slice(0, 4201)),
    },
}
ebsd_pad = {
    "0s": {1: [(0, 0), (0, 0)], 2: [(0, 0), (0, 0)], 3: [(0, 0), (0, 0)]},
    "175c": {1: [(100, 0), (0, 0)], 2: [(0, 0), (0, 0)], 3: [(0, 0), (0, 0)]},
    "300c": {1: [(0, 0), (0, 0)], 2: [(0, 0), (0, 0)], 3: [(0, 0), (0, 0)]},
    "325c": {1: [(0, 0), (0, 0)], 2: [(0, 0), (0, 0)], 3: [(100, 0), (0, 0)]},
}

Image to load

In [4]:
#data_type = "ebsd"
data_type = "bse"

Load image

In [5]:
if data_type == "ebsd":
    mask = np.load(os.path.join(dir_imreg, "mask_ebsd_correct.npy"))
    img = plt.imread(os.path.join(dir_imreg, "ebsd_correct.png"))
    intensity_img = img.copy()
else:
    img = plt.imread(os.path.join(dir_bse, "4500x_cropped2_fused_cropped_histmatch_sub.png"))
    intensity_img = plt.imread(os.path.join(dir_bse, "4500x_cropped2_fused_cropped.png"))
    if img.ndim > 2:
        img = skc.rgb2gray(img[..., :3])
    if intensity_img.ndim > 2:
        intensity_img = skc.rgb2gray(intensity_img[..., :3])
    mask = np.ones(img.shape, dtype=bool)

img = img.astype(np.float32)
figsize = (9 * img.shape[1] / max(img.shape), 3 * img.shape[0] / max(img.shape))

# Detection steps

### 1. Intensity processing

In *ImageJ*:
1. Histogram matching to 300C #1, available in the *Bleach Correction* plugin (Image > Adjust > Bleach Correction)
2. Unsharp mask, i.e. subtract a blurred copy of the image and rescale to obtain the same contrast of large structures as in the input image (Process > Filters > Unsharp Mask)
3. Subtract background (Process > Subtract Background)

In [6]:
# Normalize to enable thresholding independent of absolute intensities
img2 = img - img[mask].mean()
img3 = img2 / np.linalg.norm(img2[mask])

### 2. Generate elevation map

In [7]:
# Elevation map
elevation_map = skf.sobel(img3).astype(np.float32)

### 3. Determine markers for background and particles

In [8]:
int_range = (img3[mask].min(), img3[mask].max())
diff = abs(np.diff(int_range)[0])

# Threshold for markers
threshold = skf.threshold_triangle(img3[mask])
background = threshold + 0.20 * diff  # 0.15
particles = threshold + 0.30 * diff  # 0.25
print(threshold, background, particles)

## Intensity histogram
#fig, ax = plt.subplots()
#ax.hist(img3[mask].ravel(), bins=255, range=int_range);
#ax.set_xlabel("Intensity")
#ax.set_ylabel("Frequency")
#ax.set_xlim(int_range)
#ax.axvspan(int_range[0], background, color="C1", alpha=0.5, label="Background")
#ax.axvspan(particles, int_range[1], color="C2", alpha=0.5, label="Particles")
#ax.legend()
#fig.savefig(os.path.join(dir_partdet, f"{data_type}_hist_intensity.png"), **savefig_kw)

1.1931324e-05 0.0003617879572630045 0.0005367162741094943


### 4. Compute watershed and 5. remove holes within particles segmented from the watershed

In [20]:
# Generate sure markers of background and particles from the extreme parts of the
# intensity histogram
markers = np.zeros(img3.shape, dtype=np.uint8)
markers[img3 < background] = 1
markers[img3 > particles] = 2

# Compute the watershed transformation (computationally intensive)
segmentation = watershed(elevation_map, markers)

# Remove small holes with mathematical morphology
segmentation = ndimage.binary_fill_holes(segmentation - 1)

# Label particles
#labeled_particles, n = ndimage.label(segmentation)
#print(n)

# Inspect segmentation (useful to plot when optimizing processing and detection
# parameters)
#fig, ax = plt.subplots(ncols=3, sharex=True, sharey=True, figsize=figsize)
#ax[0].imshow(img3, cmap="gray")
#ax[1].imshow(markers)
#ax[2].imshow(img3, cmap="gray")
#ax[2].contour(segmentation, colors="r", linewidths=0.5)
#for a in ax:
#    a.axis("off")
#fig.tight_layout(pad=0.1)
#fig.savefig(os.path.join(dir_partdet, f"{data_type}_maps_segmentation.png"), **savefig_kw)

In [39]:



step_size = dict(ebsd=100, bse=1 / 39.2e-3)

scale_ebsd_bse = step_size["ebsd"] / step_size["bse"]
scale_bse_ebsd = step_size["bse"] / step_size["ebsd"]
x_bin = int(np.ceil(scale_ebsd_bse))
upscale_factor = x_bin * scale_bse_ebsd

segmentation2 = np.pad(segmentation, ebsd_pad[sample][dset_no])
segmentation3 = segmentation2[ebsd_slice[sample][dset_no]]
segmentation4 = skt.rescale(segmentation3, upscale_factor)

intensity_img2 = np.pad(intensity_img, ebsd_pad[sample][dset_no])
intensity_img3 = intensity_img2[ebsd_slice[sample][dset_no]]
intensity_img4 = skt.rescale(intensity_img3, upscale_factor)

labeled_particles, n = ndimage.label(segmentation4)
print(n)

6747


### 6. Remove incorrectly detected particles

In [ ]:
del elevation_map
del markers
del segmentation
gc.collect()

In [40]:
# Remove incorrectly detected particles
regions = MapRegions(
    label_map=labeled_particles,
    background_label=0,
    intensity_image=intensity_img4,
)
mean_intensity = regions.mean_intensity
roundness = regions.roundness
solidity = regions.solidity

In [ ]:
## Check spatial distribution of rejection properties
#titles = ["Mean intensity", "Roundness", "Solidity"]
#fig, ax = plt.subplots(ncols=3, sharex=True, sharey=True, figsize=figsize)
#ax[0].imshow(regions.get_map_data(mean_intensity))
#ax[1].imshow(regions.get_map_data(roundness))
#ax[2].imshow(regions.get_map_data(solidity))
#for i, ax_i in enumerate(ax):
#    ax_i.set_title(titles[i])
#    ax_i.axis("off")
#fig.tight_layout(pad=-0.1)
#fig.savefig(os.path.join(dir_partdet, f"{data_type}_maps_reject.png"), **savefig_kw)

In [41]:
# Filter detected particles
threshold_mean_intensity = np.percentile(mean_intensity, 55)  # 55
threshold_roundness = np.percentile(roundness[roundness > 0], 25)  # 25
threshold_solidity = np.percentile(solidity[solidity > 0], 25)  # 25
print(threshold_mean_intensity, threshold_roundness, threshold_solidity)

keep = (
    mean_intensity > threshold_mean_intensity,
    roundness == 0,
    roundness > threshold_roundness,
    solidity == 0,
    solidity > threshold_solidity,
)

# Reject some particles
regions2 = regions[np.logical_or.reduce(keep)]
print(regions2)

0.40941421687602997 0.9999999403953552 1.6190476417541504
MapRegions: 6482


In [ ]:
# Check histograms of rejection properties
titles = ["Mean intensity", "Roundness", "Solidity"]
# Inspect thresholds
fig, (ax0, ax1, ax2) = plt.subplots(ncols=3)
ax0.hist(mean_intensity, bins=100);
ax1.hist(roundness, bins=100);
ax2.hist(solidity, bins=100);
ax0.axvspan(threshold_mean_intensity, mean_intensity.max(), color="C1", alpha=0.5, label="Keep")
ax1.axvspan(threshold_roundness, roundness.max(), color="C1", alpha=0.5, label="Keep")
ax2.axvspan(threshold_solidity, solidity.max(), color="C1", alpha=0.5, label="Keep")
for i, ax in enumerate((ax0, ax1, ax2)):
    ax.set_xlabel(titles[i])
    ax.set_ylabel("Frequency")
    ax.legend()
fig.tight_layout()
fig.savefig(os.path.join(dir_partdet, f"{data_type}_hist_reject.png"), **savefig_kw)

In [42]:
# Check plot of rejected particles
with plt.rc_context(rc={"lines.linewidth": 0.5}):
    contour_kwds = dict(zorder=2, linewidths=0.5)
    fig, ax = plt.subplots(sharex=True, sharey=True, figsize=(10, 10))
    ax.imshow(regions.intensity_image, cmap="gray")
    ax.contour(~regions.is_background_map, colors="red", **contour_kwds)
    ax.contour(~regions2.is_background_map, colors="lime", **contour_kwds)
    ax.axis("off")
    fig.tight_layout()
#    fig.savefig(os.path.join(dir_partdet, f"{data_type}_maps_particles.png"), **savefig_kw)

In [43]:
data_type = "bse"

In [44]:
# Save labeled particles to file
if data_type == "ebsd":
    np.save(os.path.join(dir_partdet, "ebsd_labels_filled"), labeled_particles)
elif data_type == "bse":
    np.save(os.path.join(dir_partdet, "bse_labels_filled_filtered"), regions2.label_map)
else:  # bse_full
    np.save(os.path.join(dir_partdet, "bse_full_labels_filled_filtered"), regions2.label_map)